<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Test" data-toc-modified-id="Test-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Test</a></span><ul class="toc-item"><li><span><a href="#H" data-toc-modified-id="H-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>H</a></span></li></ul></li></ul></div>

In [ ]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
%watermark -p numpy,sklearn,pandas
%watermark -p cv2,PIL,matplotlib
%watermark -p torch,torchvision,torchaudio
%watermark -p tensorflow,tensorboard
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config IPCompleter.use_jedi = False

from IPython.display import display, Markdown, HTML, Image, Javascript
from IPython.core.magic import register_line_cell_magic, register_line_magic, register_cell_magic
display(HTML('<style>.container { width:%d%% !important; }</style>' % 90))

import sys, os, io, time, random, math
import json, base64, requests
import os.path as osp
import numpy as np

def _IMPORT_(x):
    try:
        segs = x.split(' ')
        g = globals()
        if 'github.com' in segs[1]:
            uri = segs[1].replace('github.com', 'raw.githubusercontent.com')
            mod = uri.split('/')
            for s in ['main', 'master']:
                uri = 'https://' + '/'.join(mod[:-1]) + '/main/' + mod[-1] + '.py'
                x = requests.get(uri)
                if x.status_code == 200:
                    x = x.text
                    break
        elif 'gitee.com' in segs[1]:
            mod = segs[1].split('/')
            for s in ['/raw/main/', '/raw/master/']:
                uri = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[3:]) + '.py'
                x = requests.get(uri)
                if x.status_code == 200:
                    x = x.text
                    break
        elif segs[1][0] == '/':
            with open(segs[1] + '.py') as fr:
                x = fr.read()
        exec(x, g)
    except:
        pass


In [1]:

import logging
def get_logger(name, level=logging.DEBUG, filepath=None, console=True):
    logger = logging.getLogger(name)
    logger.setLevel(level)
    formatter = logging.Formatter('%(asctime)s - %(filename)s - %(funcName)s:%(lineno)d - %(name)s - %(levelname)s - %(message)s')
    if console:
        console = logging.StreamHandler()
        console.setLevel(level)
        console.setFormatter(formatter)
        logger.addHandler(console)
    if filepath:
        filelog = logging.FileHandler(filepath)
        filelog.setLevel(level)
        filelog.setFormatter(formatter)
        logger.addHandler(filelog)
    return logger

## Test

### H

In [9]:
view_center[0]

TypeError: 'List' object does not support indexing

In [4]:
import traitlets

view_center = traitlets.List((0.5, 0.5))

In [3]:
logger.debug('11')

2021-11-05 20:11:02,625 - <ipython-input-3-4b7222c24a96> - <module>:1 - JetRep - DEBUG - 11


In [38]:
from traitlets import Int, Float, Unicode, Tuple, List, Dict
from traitlets.config.configurable import LoggingConfigurable
import traitlets

class A(LoggingConfigurable):
    latest_sorting = Dict(default_value={"field": None, "order": 'test'})
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    @traitlets.observe('latest_sorting')
    def _update_latest_sorting(self, change):
        print(change)

In [40]:
test = A()

In [41]:
test.latest_sorting =  {"field": None, "order": 'test'}

{'name': 'latest_sorting', 'old': traitlets.Undefined, 'new': {'field': None, 'order': 'test'}, 'owner': <__main__.A object at 0x7fedd082e550>, 'type': 'change'}


In [44]:
test.latest_sorting =  {"field": None, "order": 'aaa'}

{'name': 'latest_sorting', 'old': {'field': None, 'order': 'test'}, 'new': {'field': None, 'order': 'aaa'}, 'owner': <__main__.A object at 0x7fedd082e550>, 'type': 'change'}


In [50]:
a = {"field":'11', "order": 'aaa'}

In [51]:
a.get('field', 'none')

'11'

In [43]:
type(test.latest_sorting)

dict

In [52]:
class DotDict(dict):
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

    def __init__(self, *args, **kwargs):
        for arg in args:
            if isinstance(arg, dict):
                for k, v in arg.items():
                    self[k] = v
        if kwargs:
            for k, v in kwargs.items():
                self[k] = v

    def __getstate__(self):
        return self.__dict__

    def __setstate__(self, d):
        self.__dict__.update(d)

In [54]:
a = {'a': 10, 'b': 'h'}
b = DotDict(a)

In [55]:
b.a

10

In [56]:
b.h